<a href="https://colab.research.google.com/github/Nathan-Nicolau/RedesNeuraisTCC/blob/main/RedeConvolucionalTCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Sem nenhuma execução prévia ainda**

---



In [ ]:
!pip install rasterio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab as rcParams
import matplotlib

matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 14
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import seaborn as sns
plt.style.use('fivethirtyeight')
rcParams['figure.figsize'] = 18, 6
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

caminho no Drive das imagens do dataset

In [ ]:
path = 'drive/...caminho'
classList = os.listdir(path)
#verificação da quantidade de classes
print(classList)

In [ ]:
x = [] #Vetor que receberá as imaagens do Dataset
y = [] #Vetor que receberá as informações das classes
i = 0

for i in range(len(classList)):
  pathClass = os.path.join(path,classList[i])
  imgList = os.listdir(pathClass)
  for j in range(len(imgList)):
    pathImg = ps.path.join(pathClass,imgList[j])
    src = rasterio.open(pathImg)
    im = src.read()
    im = im.transpose([1,2,0])
    im1 = im[:,:,1:9]
    im2 = im[:,:,11:]
    im = np.dstack((im1,im2))
    print(str(j) + ' ' +classList[i])
    y.append(i)
    x.append(im)

In [ ]:
#Visualização do "shape" dos dados
x = np.array(x)
y = np.array(y)

In [ ]:
#Uma forma de verificação opicional da razão de quantidade de imagens
#pela quantidade de classes existentes

classe, qtd = np.unique(y,return_counts=True)
df = pd.DataFrame(data=qtd,index=classList, columns=['Amostras'])
ax = sns.barplot(x = df.index, y="Amostras", data=df)